# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [8]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [9]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [10]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/1

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [11]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [12]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [13]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/

In [14]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [15]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]}

In [16]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [17]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 4 relevant links


{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [18]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


{'links': [{'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'status page', 'url': 'https://status.huggingface.co/'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Discord community', 'url': 'https://huggingface.co/join/discord'},
  {'type': 'API endpoints', 'url': 'https://endpoints.huggingface.co'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [19]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [20]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 6 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
moonshotai/Kimi-K2.5
Updated
about 3 hours ago
•
25.4k
•
1.11k
nvidia/personaplex-7b-v1
Updated
1 day ago
•
54.5k
•
1.46k
Tongyi-MAI/Z-Image
Updated
1 day ago
•
1.92k
•
663
Qwen/Qwen3-TTS-12Hz-1.7B-CustomVoice
Updated
1 day ago
•
181k
•
762
deepseek-ai/DeepSeek-OCR-2
Updated
about 8 hours ago
•
45.3k
•
530
Browse 2M+ models
Spaces
Running
on
Zero
Featured
1.01k
Qwen3-TTS Demo
🎙
1.01k
Transform text into natural-sounding speech with custom voices
Running
on
Zero
Featured
1.26k
Qwen Image Multiple Angles 3D Camera
🎥
1.26k
Adjust camera 

In [21]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [23]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nmoonshotai/Kimi-K2.5\nUpdated\nabout 3 hours ago\n•\n25.4k\n•\n1.11k\nnvidia/personaplex-7b-v1\nUpdated\n1 day ago\n•\n54.5k\n•\n1.46k\nTongyi-MAI/Z-Image\nUpdated\n1 day ago\n•\n1.92k\n•\n663\nQwen/Qwen3-TTS-12Hz-1.7B-CustomVoice\nUpdated\n1 day ago\n•\n181k\n•\n762\ndeepseek-ai/DeepSeek-OCR-2\nUpdated\nabout 8 hours ago\n•\n45.3k\n•\n530\nBrowse 2M+ models\nSpaces\nRunning\non\nZero\nFeat

In [25]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [26]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 15 relevant links


# Hugging Face: The AI Community Building the Future

## About Hugging Face
Hugging Face is a pioneering AI and machine learning (ML) platform serving as the global community hub where innovators, researchers, and developers collaborate to create the future of AI. It provides an open and ethical space for sharing, discovering, and experimenting with machine learning models, datasets, and applications.

## Platform Highlights
- **Extensive Model Repository:** Home to over 2 million ML models spanning text, image, video, audio, and even 3D modalities. Users can browse, test, and integrate models such as text-to-speech, image generation, and OCR.
- **Massive Dataset Collection:** Contains 500,000+ datasets uploaded by a rich and diverse community, facilitating high-quality, large-scale training and evaluation.
- **Spaces:** A unique feature that hosts ML applications and demos that users can interact with directly online, showcasing novel AI capabilities.
- **Open-Source Stack:** Hugging Face supports rapid innovation through a robust open-source ecosystem, powering faster development and decentralizing machine learning.
- **Community and Collaboration:** The platform fosters co-creation by enabling unlimited hosting and collaboration on public models, datasets, and apps, forming a vibrant community of ML engineers, scientists, and AI enthusiasts.

## Company Culture and Values
- **Open & Ethical AI:** Hugging Face emphasizes openness, transparency, and ethics in the AI community to build trustworthy and responsible AI technologies.
- **Collaborative Innovation:** The company nurtures an inclusive environment where knowledge-sharing and teamwork drive progress.
- **Empowerment:** Hugging Face empowers users at all skill levels—from beginner developers to expert researchers—to learn, grow, and contribute to the AI frontier.
- **Community-Driven:** A fast-growing community underpins continuous improvements and diverse innovations across all AI modalities.

## Customers & Use Cases
Hugging Face serves a wide spectrum of users:
- Individual machine learning engineers building portfolios or experimenting with new models.
- Academic and research organizations sharing and accessing cutting-edge datasets and pre-trained models.
- Enterprises leveraging Hugging Face’s open source and commercial solutions to accelerate AI adoption and customization.
- AI startups and developers deploying applications for speech recognition, natural language processing, image generation, and more.

## Careers & Opportunities
Join Hugging Face if you want to be part of shaping the future of artificial intelligence!
- Work alongside passionate AI experts and contribute to impactful open-source projects.
- Thrive in a dynamic, innovative culture promoting continuous learning and community engagement.
- Opportunities likely span software engineering, research science, community management, and enterprise support roles.
- Embrace remote-friendly work structures and a mission-driven environment.

## Connect & Explore
- Access 2M+ models, 500k+ datasets, and 1M+ AI applications on the platform.
- Join the vibrant Hugging Face community and start collaborating today.
- Learn more and explore AI apps at [huggingface.co](https://huggingface.co).

---

Hugging Face is shaping a future where AI is accessible, collaborative, and ethical—making machine learning innovation open to everyone. Join the community building the future of AI!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [27]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [28]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face Brochure

---

## Who We Are

**Hugging Face** is a vibrant AI community dedicated to building the future of machine learning (ML). We are a collaboration platform where the ML community worldwide connects to create, share, and innovate using open-source models, datasets, and applications across modalities like text, image, video, audio, and even 3D.

Our mission is to empower ML engineers, scientists, and end users to learn, collaborate, and share their work, fostering an open, ethical AI future.

---

## Our Platform & Offerings

- **Models:** Access and contribute to over 2 million openly available ML models that power countless innovative AI applications.
- **Datasets:** Discover from half a million datasets updated regularly for diverse research and development needs.
- **Spaces:** Host and explore AI apps and demos such as natural-sounding text-to-speech, 3D image controls, and image generation tools.
- **Community:** Join a fast-growing global community collaborating daily to push the boundaries of AI.
- **Open Source Stack:** Leverage our powerful open source tools for faster, flexible, and scalable AI development.
- **Enterprise Solutions & Pricing:** Tailored options that bring the best of Hugging Face to businesses of all sizes.

---

## Why Choose Hugging Face?

- Centralized hub to **share, explore, and experiment** with open ML resources.
- Support across all **modalities** (text, image, audio, video, 3D).
- Deep integration with the latest AI advancements from leading AI researchers and companies.
- **Build and showcase your portfolio** by contributing your models and datasets.
- Ethically-driven commitment to **open and inclusive AI** development.

---

## Our Community & Culture

At Hugging Face, collaboration is at our core. We cultivate an inclusive, creative, and innovative culture that:

- Encourages **knowledge sharing** and open communication.
- Supports both newcomers and experts through accessible tools and learning resources.
- Values ethical AI practices ensuring technology benefits everyone.
- Promotes a fast-paced but welcoming environment fostering growth and impact.

---

## Career Opportunities

Join Hugging Face to be part of a transformative mission in AI!

Positions include:
- Machine Learning Engineers
- Research Scientists
- Software Developers
- Community Managers
- Product and Design Roles

If you are passionate about open source, AI ethics, and building next-gen AI tooling that impacts millions, Hugging Face offers a dynamic workplace to grow your career while shaping the future of AI.

---

## Join Us

Explore AI applications, browse millions of models and datasets, or start sharing your own ML projects today at [huggingface.co](https://huggingface.co).

Become part of the AI community that is defining tomorrow — with Hugging Face.

---

### Brand Colors
- Yellow: #FFD21E
- Orange: #FF9D00
- Gray: #6B7280

---

*Hugging Face – The AI community building the future.*

### The code below is to complement Week 2 Day 2 Exercise - Using Gradio

In [ ]:
import gradio as gr

name_input = gr.Textbox(label="Company name:")
url_input = gr.Textbox(label="Landing page URL including https://")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_brochure,
    inputs=[name_input, url_input],
    outputs=[message_output],
    examples=[
        ["HuggingFace", "https://huggingface.co"]
    ],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the AI community at the forefront of building the future of machine learning. It serves as a collaboration platform where machine learning engineers, scientists, and enthusiasts come together to create, share, and explore models, datasets, and applications across multiple AI modalities including text, images, video, audio, and 3D.

The platform hosts over 2 million models, 500,000 datasets, and more than 1 million applications, making it a vibrant hub for innovation and experimentation in open-source machine learning.

---

## Our Mission

Hugging Face’s mission is to democratize machine learning by empowering the next generation of AI practitioners to learn, collaborate, and share their work openly and ethically. The goal is to build an open, inclusive, and ethical AI future—one commit at a time.

---

## What We Offer

### The Hugging Face Hub
- A centralized place to **discover, share, and experiment** with open-source ML resources.
- Unlimited hosting and collaboration on public models, datasets, and applications.
- Fast development with access to powerful open-source ML libraries and tools.

### Spaces
- An easy way to **run and demo AI applications live** on the web.
- Showcases include text-to-speech, 3D image manipulation, OCR, and image editing demos powered by advanced models.

### Community & Collaboration
- A fast-growing global community of AI researchers, engineers, and developers.
- Active forums, collaboration on cutting-edge projects, and a rich ecosystem of shared knowledge.
- Regular updates, scientific papers, and thoughtful articles fostering learning and innovation.

---

## Our Culture

- **Open and Ethical AI:** Strong emphasis on transparency, openness, and responsible AI development.
- **Community-Driven:** Collaboration is at our core—sharing code, data, and ideas drives progress.
- **Inclusive Learning:** Providing educational resources and supporting career growth for ML practitioners at all levels.
- **Innovation at Scale:** Encouraging experimentation through state-of-the-art tools and a vibrant ecosystem.

---

## Customers & Use Cases

Hugging Face serves a wide range of users including:
- Machine Learning Engineers and Data Scientists building scalable AI solutions.
- Researchers pushing the boundaries of AI across modalities.
- Enterprises integrating AI capabilities into business operations via Hugging Face Enterprise solutions.
- Enthusiasts and learners tapping into the rich open-source resources to develop machine learning skills.

---

## Careers at Hugging Face

- We are a growing team of over 190 members passionate about AI.
- Open roles across engineering, research, product, and community engagement.
- Join us to contribute to the AI revolution and democratize machine learning.
- Culture values collaboration, innovation, and impact-driven work — "democratizing good machine learning, one commit at a time."
  
Visit the [Hugging Face Careers Page](https://huggingface.co/company/careers) to explore opportunities.

---

## Branding & Identity

- Brand colors: Bright Yellow (#FFD21E), Vibrant Orange (#FF9D00), and Neutral Gray (#6B7280).
- Logos available in multiple formats (.svg, .png, .ai) for consistent branding across platforms.
- Hugging Face’s recognizable smiley face logo symbolizes openness and community spirit.

---

## Connect with Us

- Website: [huggingface.co](https://huggingface.co)
- GitHub: github.com/huggingface
- Twitter: twitter.com/huggingface
- LinkedIn: linkedin.com/company/hugging-face
- Discord: Join the community chat for real-time collaboration

---

**Hugging Face** — Building the future of AI *together.*  
Join us and be part of the machine learning revolution.

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>